In [ ]:
###
# 
#
#
#
#

In [1]:
import os
import ee
import geemap
import gdal
from samgeo import SamGeo

In [3]:
### view the potential basemap layers 
# basemaps = geemap.basemaps.keys()
# print(basemaps)

In [4]:
# define the map object 
Map = geemap.Map()
# single pane map with drawing tools 
Map


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
# convert the features drawn on the map into ee objects 
roi = ee.FeatureCollection(Map.draw_features)
roi

In [9]:
# split map option --- good for a validation of the points maybe after someone has created it. 
# Map2 = geemap.Map()
# Map2.split_map(left_layer='USGS NAIP Imagery', right_layer='Google Satellite')
# #call map object to visualize
# Map2

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [4]:
# add spatial data from local environment

In [9]:
# Pull in Naip Based on the ROI map
naip = ee.ImageCollection("USDA/NAIP/DOQQ").filter(ee.Filter.date('2014-01-01', '2015-12-31')).filterBounds(roi).mosaic().clip(roi)
# create an NDVI band 
ndvi = naip.normalizedDifference(['N', 'R']).rename('NDVI')
# add layer to back to naip feature
naip.addBands(ndvi)

## eventually want to develop a texture layer on the green and nir bands. The focal area should be representative of the expect tree crown area
## https://www.azavea.com/blog/2016/09/14/multispectral-1m-naip-imagery-model-urban-forest/


In [11]:
# visualization options 
rgbVisBandSpec = {'bands' : ['R','G','B'],
                 'min' : 0,
                  'max' : 255} # lower this value to bring more brightness to the image. 
nirFalseColor = {'bands' : ['N','G','B'],
                 'min' : 0,
                  'max' : 255}
ndviColor = {'bands':['NDVI'],
             'min': -1,
             'max': 1, 
             'palette': ['blue', 'white', 'green']}


In [12]:
aoiMap = geemap.Map()
# aoiMap.addLayer(naip, rgbVisBandSpec, "NAIP RGB").addLayer(naip, nirFalseColor, "NAIP NIR False Color")
aoiMap.addLayer(naip, rgbVisBandSpec, "NAIP RGB")
aoiMap.addLayer(naip, nirFalseColor, "NAIP NIR False Color")
# aoiMap.addLayer(naip, ndviColor, "NAIP NDVI")
aoiMap.centerObject(roi, 12)
aoiMap

Map(center=[40.155879863782644, -99.69268799999915], controls=(WidgetControl(options=['position', 'transparent…

In [42]:
### GEE specific clustering --- need some work to understand the process but it runs fast at this scale 
n2 = ee.Algorithms.Image.Segmentation.SNIC(naip)
## cluster visualization 
Map.addLayer(n2)
Map

Map(bottom=393369.0, center=[41.01798769755012, -98.16232681274415], controls=(WidgetControl(options=['positio…

In [34]:
### gpu rendering --- need to work it out. 
### https://pytorch.org/get-started/locally/ --- amd gpu are not support on windows. 
import torch
to
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [43]:
### testing the segment geospatial library 
# # export image 

### example from https://geemap.org/notebooks/134_ee_to_geotiff/
point = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(point)
    .filterDate('2008-01-01', '2018-01-01')
    .filter(ee.Filter.listContains("system:band_names", "N"))
)
image = collection.first()
Map.addLayer(image)
Map


Map(bottom=393369.0, center=[41.01798769755012, -98.16232681274415], controls=(WidgetControl(options=['positio…

In [47]:
geemap.ee_to_geotiff(naip, 'naip1.tif', zoom=15, vis_params={'bands': ['N', 'R', 'G']})


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\dune\AppData\Local\R-MINI~1\envs\agroforestry\lib\site-packages\geemap\common.py", line 15111, in tms_to_geotiff
    draw_tile(source, south, west, north, east, zoom, output, quiet, **kwargs)
  File "C:\Users\dune\AppData\Local\R-MINI~1\envs\agroforestry\lib\site-packages\geemap\common.py", line 15046, in draw_tile
    x0, y0 = deg2num(lat0, lon0, zoom)
  File "C:\Users\dune\AppData\Local\R-MINI~1\envs\agroforestry\lib\site-packages\geemap\common.py", line 14970, in deg2num
    ytile = (1 - math.log(math.tan(lat_r) + 1 / math.cos(lat_r)) / math.pi) / 2 * n
ValueError: math domain error

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\dune\AppData\Local\R-MINI~1\envs\agroforestry\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\dune\AppData\Local\Temp\ipykernel_6848\9

In [36]:
%%time
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    device=None,
    sam_kwargs=None,
)
sam

CPU times: total: 9.44 s
Wall time: 3.59 s


In [37]:
%%time
sam.generate("naip.tif", output="masks.tif", foreground=True, unique=True)

KeyboardInterrupt: 

In [ ]:
%%time
sam.show_masks(cmap="binary_r")

In [ ]:
%%time
sam.show_anns(axis="off", alpha=1, output="annotations.tif")